In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep

In [ ]:
soups = []

categories = [
    ["http://www.medias-presse.info/category/international/",402],
    ["http://www.medias-presse.info/category/societe/",298],
    ["http://www.medias-presse.info/category/culture/",112],    
]

counter = 0

for category, max_n in categories:
    for n in range(1,max_n):
        r  = requests.get(category+"page/{}/".format(n))
        data = r.text
        soup = BeautifulSoup(data, "lxml")

        soups.append(soup)
        sleep(0.1)
        
        counter += 1
        if counter % 10 == 0:
            print(counter, "pages completed.")

10 pages completed.
20 pages completed.
30 pages completed.
40 pages completed.
50 pages completed.
60 pages completed.
70 pages completed.


In [ ]:
all_links = []

for soup in soups:
    ls = []

    for link in soup.find_all('a'):
        url = link.get('href')
        x = url.split("/")
        if len(x) > 3:
            last_value = x[-2]
            last_value2 = x[-1]
            
            if last_value.isdigit():
                last_value = int(last_value)
                if last_value > 0 and last_value < 90000 and last_value2 != "#comments":
                    ls.append(url)
    
    all_links.extend(ls)

all_links = list(set(all_links))
print(len(all_links))

In [ ]:
di_big = {"id":[], "agency": [], "url": [], "title": [], "text":[], "author":[]}
counter = 0

for link in all_links:
    sleep(0.1)
    r  = requests.get(link)
    data = r.text
    soup = BeautifulSoup(data, "lxml")

    paragraphs_ls = []

    for paragraph in soup.find_all('p', style="text-align: justify;"):
        paragraphs_ls.append(paragraph.text)

    paragraphs = "\n".join(paragraphs_ls)
    
    _id = link.split("/")[-2]
    author = soup.find('a', rel='author').text
    title = soup.title.text
    
    if paragraphs != "":
        di_big["id"].append(_id)
        di_big["agency"].append("media-presse.info")
        di_big["url"].append(link)
        di_big["title"].append(title)
        di_big["text"].append(paragraphs)
        di_big["author"].append(author)
    
    counter += 1
    if counter%250==0:
        print(counter, "articles processed.")

In [ ]:
df = pd.DataFrame.from_dict(di_big)
print(df.shape)
df.to_csv("fakenews_big2.csv", index=False)